In [2]:
import sys, os, time
os.environ['FOUNDATION_RUN_MODE'] = 'jupyter'
# %load_ext autoreload
# %autoreload 2
# %pdb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.multiprocessing as mp
import torchvision.models
from torch.utils.data import Dataset, DataLoader
import gym
import inspect
import numpy as np
%matplotlib notebook
#%matplotlib tk
import matplotlib.pyplot as plt
#plt.switch_backend('Qt5Agg') #('Qt5Agg')
import foundation as fd
from foundation import models
from foundation import util
from foundation import train
#from foundation.util import replicate, Cloner

import project as proj

np.set_printoptions(linewidth=120)

In [4]:
x = np.random.randn(10)


In [3]:
root = os.environ['FOUNDATION_SAVE_DIR']
names = [n for n in os.listdir(root) if True]
for i, name in enumerate(names):
    print(i,name)

0 test-cmd_191121-223836
1 test-cmd_191123-013853
2 test-cmd_191123-014508
3 test-cmd_191123-104827
4 test-cmd_191123-105743
5 test-cmd_191123-110022
6 test-cmd_191123-110320
7 test-cmd_191123-110650
8 test-cmd_191123-110727
9 test-cmd_191123-111846
10 test-pycharm_191121-222639
11 test-pycharm_191121-225857
12 test-pycharm_191123-005824
13 test-pycharm_191123-013319
14 test-pycharm_191123-013427
15 test-pycharm_191123-013626
16 test-pycharm_191123-111037
17 test-pycharm_191123-111801


In [11]:
if True:
    name = names[0]
    path = os.path.join(root, name)
    A = None
    print('Using: {}'.format(name))
else:
    path = None
    A = train.get_config('model')
    print('Using: blank')

Using: test-cmd_191121-223836


In [13]:
A, (dataset, *other), model = train.load(path=path, A=A, get_model=proj.get_model, get_data=proj.get_data)
print(model)
print(model.optim)

Found 0 checkpoints. However, a config file was found
Loaded /mnt/c/Users/anwan/Documents/workspace/trained_nets/test-cmd_191121-223836
Dataset mnist moved to cpu
Dataset din=(1, 28, 28), dout=10
input_dim is an alias for din
input_dim default: (1, 28, 28)
output_dim is an alias for dout
output_dim default: 10
nonlin default: prelu
output_nonlin default: None
criterion default: cross-entropy
Model(
  (net): Sequential(
    (0): Flatten()
    (1): Linear(in_features=784, out_features=300, bias=True)
    (2): PReLU(num_parameters=1)
    (3): Linear(in_features=300, out_features=10, bias=True)
  )
  (criterion): CrossEntropyLoss()
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)


In [14]:
A.dataset.batch_size = 64
loader = iter(train.get_loaders(dataset, batch_size=A.dataset.batch_size, num_workers=A.num_workers,
                            shuffle=True, drop_last=False, ))

In [15]:
batch = next(loader)
batch = util.to(batch, A.device)

In [16]:
X, Y = batch
X.shape, Y.shape

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [1]:
import omnifig as fig

In [2]:
fig.initialize()

In [1]:
import omnidata as od

In [26]:
from tabulate import tabulate
def comp(base, *objs, use_repr=False, mlen=80):
    sfn = repr if use_repr else str
    full = set(dir(base))
    for obj in objs:
        full = full.union(dir(obj))
    table = [(k, sfn(getattr(base, k, None))[:mlen], *[sfn(getattr(obj, k, None))[:mlen] for obj in objs]) for k in full]
    print(tabulate(table))
from omnibelt import tricks as trx

In [27]:
M = None

In [28]:
class A(trx.auto_methods, inheritable_auto_methods=['f', 'g', 'h']):

    @staticmethod
    def _fix_missing_args(missing, src, method, args, kwargs):
        print(missing, src, method, args, kwargs)
        return args, kwargs


    def _auto_fix_args(self, src, method, args, kwargs) -> None:
        global M
        M = method
        print(src, method, type(method), args, kwargs)
        return super()._auto_fix_args(src, method, args, kwargs)
        pass

    def f(self, y, x=10):
        print('f', x)

    def f2(self, x=100):
        print('f2', x)

    def f3(y, x=100):
        print('f2', x)

    @classmethod
    def g(cls, y, x=-1):
        print('g', x, y, cls)

    @classmethod
    def g2(cls, y, x=-1):
        print('g2', x, y, cls)

    @staticmethod
    def h(y, x=-1):
        print('h', x, y)

    @staticmethod
    def h2(y, x=-1):
        print('h2', x, y)



f <function A.f at 0x7fab54b97040>
g <bound method A.g of <class '__main__.A'>>
h <function A.h at 0x7fab54b97430>


In [30]:
dir(A)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_fix_args',
 '_auto_method_arg_fixer',
 '_auto_methods',
 '_fix_missing_args',
 'captured_method_call',
 'f',
 'f2',
 'f3',
 'g',
 'g2',
 'h',
 'h2']

In [29]:
print(comp(A.f.fn, A.h.fn))

-----------------  --------------------------------------------------------------------------------  --------------------------------------------------------------------------------
__module__         __main__                                                                          __main__
__reduce_ex__      <built-in method __reduce_ex__ of function object at 0x7fab54b97040>              <built-in method __reduce_ex__ of function object at 0x7fab54b97430>
__delattr__        <method-wrapper '__delattr__' of function object at 0x7fab54b97040>               <method-wrapper '__delattr__' of function object at 0x7fab54b97430>
__get__            <method-wrapper '__get__' of function object at 0x7fab54b97040>                   <method-wrapper '__get__' of function object at 0x7fab54b97430>
__hash__           <method-wrapper '__hash__' of function object at 0x7fab54b97040>                  <method-wrapper '__hash__' of function object at 0x7fab54b97430>
__reduce__         <built-in method __

In [11]:
dir(A.h.fn)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [19]:
A.f3

<function __main__.A.f3(y, x=100)>

In [18]:
type(A.f3), type(A.f.fn), type(A.h.fn)

(function, function, function)

In [16]:
A.h.fn.__class__

function

In [54]:
a = A()

In [39]:
a.f()

<class '__main__.A'> <function A.f at 0x7f26d5357700> <class 'function'> () {}
[<Parameter "y">] <class '__main__.A'> <function A.f at 0x7f26d5357700> [<__main__.A object at 0x7f26d5962a30>] {}


TypeError: f() missing 1 required positional argument: 'y'

In [48]:
a.g()

<class '__main__.A'> <bound method A.g of <class '__main__.A'>> <class 'method'> () {}
g -1 <__main__.A object at 0x7f26d5249a60> <class '__main__.A'>


In [45]:
a.h()

<class '__main__.A'> <function A.h at 0x7f26d53221f0> <class 'function'> () {}
h -1 <__main__.A object at 0x7f26d5249a60>


In [55]:
A.g2(1)

g2 -1 1 <class '__main__.A'>


In [56]:
A.g2

<bound method A.g2 of <class '__main__.A'>>

In [57]:
A.g()

AssertionError: no instance to call method on

In [49]:
M

<bound method A.g of <class '__main__.A'>>

In [37]:
dir(M)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [38]:
M.__self__

__main__.A